<a href="https://colab.research.google.com/github/KOFIYEB/Chatbox_Zephyr7B/blob/main/chatbox_Zephyr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Chatbox with Zephyr 7B**

---



We will develop a chatbot using **Zephyr 7B**, a well-regarded conversational model that has been enhanced through fine-tuning with Mistral 7B. Crafted by the Hugging Face H4 team, Zephyr 7B Beta has demonstrated superior performance compared to GPT-3.5, LLama 2 70B, and other models in the 7B size category, as evidenced by the Alpaca Leaderboard rankings. Additionally, Zephyr 7B offers the advantage of being significantly more compact, requiring only 4% of the storage space needed for GPT-3.5.

# **Load Packages**

In [ ]:
# You only need to run this once, even if you re-start the kernel or machine
# Remove the -q to print install output
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U accelerate jupyter ipywidgets

# **Load Model**

In this section, we initialize the model. You're welcome to substitute it with any conversational or instructional model from Hugging Face, ensuring that you adjust the "roles" accordingly as outlined below. However, this model is for now highly recommended.

In [2]:
models_dict = {
    "Zephyr 7B Alpha": "HuggingFaceH4/zephyr-7b-alpha",
    "Zephyr 7B Beta": "HuggingFaceH4/zephyr-7b-beta",
    # You can add more chat/instruct models here
}

model_id = models_dict["Zephyr 7B Beta"]

In [ ]:
import torch
from transformers import pipeline

# Running this may take a minute
pipe = pipeline("text-generation", model=model_id, torch_dtype=torch.bfloat16, device_map="auto")

# **Define the role**

specify the role you want the chatbot to adopt. For this example, I'll configure it to function as a sports analyst.

In [4]:
job_description = "You are a sport analyst expert. Please answer my questions."

Here we execute the model! To conclude the question-answering session, type "stop" or "exit". Feel free to disregard the UserWarning about changes to the pre-trained model configuration.

In [7]:
messages = [
    {
        "role": "system",
        "content": job_description,
    },
]

exit_terms = ["stop", "exit"]

while True:
    question = input("\nQuestion:")
    if question.lower() in exit_terms:
        break
    messages.append({"role": "user", "content": question})
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=512, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    output = outputs[0]["generated_text"]
    messages.append({"role": "assistant", "content": output})
    response_start = output.rfind('<|assistant|>')
    print(output[response_start + len('<|assistant|>'):])


Question: Rank the top 10 basketball players of all time


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



1. Michael Jordan - Six-time NBA champion, five-time MVP, and considered by many as the greatest basketball player of all time.

2. LeBron James - Four-time NBA champion, four-time MVP, and widely regarded as one of the most talented and versatile players in NBA history.

3. Bill Russell - Eleven-time NBA champion and five-time MVP, widely recognized as one of the best defenders and rebounders in NBA history.

4. Kareem Abdul-Jabbar - Six-time NBA champion and six-time MVP, holds the NBA's all-time scoring record and is widely regarded as one of the greatest centers of all time.

5. Wilt Chamberlain - Three-time NBA champion, four-time MVP, and holds numerous NBA records, including most points in a single game.

6. Magic Johnson - Five-time NBA champion and three-time MVP, widely regarded as one of the most talented and versatile players in NBA history.

7. Larry Bird - Three-time NBA champion and three-time MVP, widely regarded as one of the best shooters and all-around players in NB

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



It's a subjective question, as the title of "greatest basketball player of all time" is a matter of personal opinion. However, here are some factors that could potentially help Lebron James surpass Michael Jordan:

1. Win more NBA championships: Jordan won six NBA championships, and while James has four, he's still actively playing and has a chance to add to his total.

2. Win more NBA Finals MVP awards: Jordan won the Finals MVP award six times, while James has won it four times. More Finals MVP awards could help James' case for being the best.

3. Lead his team to more playoff series victories: Jordan led his teams to an impressive 63 playoff series victories, while James has 65. More playoff series victories could further solidify James' legacy.

4. Set more NBA records: Jordan holds numerous NBA records, but James has already broken some of them and continues to break new ones. Setting more records could help James surpass Jordan.

5. Achieve more individual accolades: Jordan won 

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



It's difficult to predict which current players will eventually make it into the top 10 all-time list as it's a subjective ranking that's based on a player's entire career, not just their current form. However, here are a few current players who have the potential to make a strong case for being among the all-time greats:

1. LeBron James - As I mentioned earlier, James is already in the top 10 list, and he's still in the prime of his career. If he continues to win championships, individual awards, and break records, there's no doubt that he'll climb higher on the list.

2. Stephen Curry - Curry has already won two NBA MVP awards and led his team to a championship in 2015. He's also one of the most prolific scorers in NBA history and has set numerous records for three-point shooting. If he can lead his team to more championships and continue to dominate the league, he could potentially crack the top 10.

3. Kevin Durant - Durant has won two NBA championships and has been named Finals 

**It worked absolutely fine, you can also create your version of ChatGPT 😲**